# License
    IPython notebook for simulating the linear wave equation with OpenCL
    Copyright (C) 2015 Andre.Brodtkorb@ifi.uio.no

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Lets have opencl ipython integration enabled
%load_ext pyopencl.ipython_ext

#Import packages we need
import numpy as np
import pyopencl as cl
import os
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

In [2]:
#Setup easier to use compilation of OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_CTX"] = "0"
os.environ["CUDA_CACHE_DISABLE"] = "1"

In [3]:
#Create OpenCL context
cl_ctx = cl.create_some_context()

#Create an OpenCL command queue
cl_queue = cl.CommandQueue(cl_ctx)

# Linear Wave Equation in 2D
The acoustic wave equation in 2D can be written
$$
\begin{align}
\frac{\partial^2 u}{\partial t^2} &= c^2 \nabla^2 u\\
&= c^2 \left[ \frac{\partial^2 u}{\partial x^2} +  \frac{\partial^2 u}{\partial y^2} \right]
\end{align}
$$
where $u$ is the string position, and c is a material specific constant. 

By approximating the temporal derivative with a backward difference, and the spatial derivative with a central difference, we get
$$
\frac{1}{\Delta t^2} (u_{i, j}^{n+1} - 2u_{i, j}^{n} + u_{i, j}^{n-1}) 
= c \left [
\frac{1}{\Delta x^2}(u_{i-1, j}^n - 2u_{i, j}^n + u_{i+1, j}^n)
+ \frac{1}{\Delta y^2}(u_{i, j-1}^n - 2u_{i, j}^n + u_{i, j+1}^n)
\right]
$$
and gathering $u^n+1$ on the left hand side and $u^n$on the right, we write
$$
u^{n+1}_{i,j} = 2u_{i,j}^n - u_{i, j}^{n-1}
+ \frac{\kappa\Delta t^2}{\Delta x^2}(u_{i-1, j}^n - 2u_{i, j}^n + u_{i+1, j}^n)
+ \frac{\kappa\Delta t^2}{\Delta y^2}(u_{i, j-1}^n - 2u_{i, j}^n + u_{i, j+1}^n)
$$
This discretization is unstable if the following CFL condition is not met
$$
\frac{1}{2} \gt \frac{\kappa\Delta t}{\Delta x^2}, \qquad
\frac{1}{2} \gt \frac{\kappa\Delta t}{\Delta y^2}
$$
or 
$$
\Delta t \lt \text{min}\left(\frac{\Delta x^2}{2\kappa}, \frac{\Delta y^2}{2\kappa}\right)
$$

In [4]:
%%cl_kernel 
__kernel void linear_wave_2D(__global float* u2, global const float* u1, __global const float* u0, float kappa, float dt, float dx, float dy) {
    //Get total number of cells
    int nx = get_global_size(0); 
    int ny = get_global_size(1);

    //Get position in grid
    int i = get_global_id(0); 
    int j = get_global_id(1); 

    //Calculate the four indices of our neighboring cells
    int center = j*nx + i;
    int north = (j+1)*nx + i;
    int south = (j-1)*nx + i;
    int east = j*nx + i+1;
    int west = j*nx + i-1;

    //Internal cells
    if (i > 0 && i < nx-1 && j > 0 && j < ny-1) {
        u2[center] = 2.0f*u1[center] - u0[center]
            + kappa*dt/(dx*dx) * (u1[west] - 2*u1[center] + u1[east])
            + kappa*dt/(dy*dy) * (u1[south] - 2*u1[center] + u1[north]);
    }
}

__kernel void linear_wave_2D_bc(__global float* u) {
    //Get total number of cells
    int nx = get_global_size(0); 
    int ny = get_global_size(1);

    //Get position in grid
    int i = get_global_id(0); 
    int j = get_global_id(1); 

    //Calculate the four indices of our neighboring cells
    int center = j*nx + i;
    int north = (j+1)*nx + i;
    int south = (j-1)*nx + i;
    int east = j*nx + i+1;
    int west = j*nx + i-1;

    if (i == 0) {
        u[center] = u[east];
    }
    else if (i == nx-1) {
        u[center] = u[west];
    }
    else if (j == 0) {
        u[center] = u[north];
    }
    else if (j == ny-1) {
        u[center] = u[south];
    }
}

In [5]:
"""
Class that holds data for the heat equation in OpenCL
"""
class LinearWaveDataCL:
    """
    Uploads initial data to the CL device
    """
    def __init__(self, u0, u1):
        #Make sure that the data is single precision floating point
        assert(np.issubdtype(u1.dtype, np.float32))
        assert(np.issubdtype(u0.dtype, np.float32))
        assert(not np.isfortran(u0))
        assert(not np.isfortran(u1))
        assert(u0.shape == u1.shape)

        #Find number of cells
        self.nx = u0.shape[1]
        self.ny = u0.shape[0]
        
        mf = cl.mem_flags 
        
        #Upload data to the device
        self.u0 = cl.Buffer(cl_ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=u0)
        self.u1 = cl.Buffer(cl_ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=u1)
        
        #Allocate output buffers
        self.u2 = cl.Buffer(cl_ctx, mf.READ_WRITE, u0.nbytes)
        
    """
    Enables downloading data from CL device to Python
    """
    def download(self):
        #Allocate data on the host for result
        u1 = np.empty((self.ny, self.nx), dtype=np.float32)
        
        #Copy data from device to host
        cl.enqueue_copy(cl_queue, u1, self.u1)
        
        #Return
        return u1;

In [6]:
"""
Computes a solution to the linear wave equation equation using an explicit finite difference scheme with OpenCL
"""
def opencl_linear_wave(cl_data, c, dx, dy, dt, nt):
    #Loop through all the timesteps
    for i in range(0, nt):
        #Execute program on device
        linear_wave_2D(cl_queue, (nx, ny), None, \
                       cl_data.u2, cl_data.u1, cl_data.u0, \
                       np.float32(c), np.float32(dt), np.float32(dx), np.float32(dy))
        linear_wave_2D_bc(cl_queue, (nx, ny), None, cl_data.u2)
        
        #Swap variables
        cl_data.u0, cl_data.u1, cl_data.u2 = cl_data.u1, cl_data.u2, cl_data.u0

In [20]:
#Create test input data
c = 1.0
nx, ny = 50, 25
dx = 1.0
dy = 2.0
dt = 0.1 * min(dx / (2.0*c), dy / (2.0*c))

u0 = np.zeros((ny, nx)).astype(np.float32)
for j in range(ny):
    for i in range(nx):
        x = (i - nx/2.0) * dx
        y = (j - ny/2.0) * dy
        if (np.sqrt(x**2 + y**2) < 10*min(dx, dy)):
            u0[j, i] = 1.0
u1 = u0
cl_data = LinearWaveDataCL(u0, u1)


#Plot initial conditions
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
fig.suptitle("Wave equation 2D", fontsize=18)

max_x = cl_data.nx*dx
max_y = cl_data.ny*dy

y, x = np.mgrid[0:max_y:dy, 0:max_x:dx]
surf_args=dict(cmap=cm.coolwarm, shade=True, vmin=0.0, vmax=1.0, cstride=1, rstride=1)
ax.plot_surface(x, y, u0, **surf_args)
ax.set_zlim(-0.5, 5.0)

    
def animate(i):
    timesteps_per_plot=5
    
    #Simulate
    if (i>0):
        opencl_linear_wave(cl_data, c, dx, dy, dt, timesteps_per_plot)

    #Download data
    u1 = cl_data.download()

    #Plot
    ax.clear()
    ax.plot_surface(x, y, u1, **surf_args)
    ax.set_zlim(-5.0, 5.0)
        

anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close(anim._fig)
anim